In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # 解決PLT錯誤，若沒有kernal意外損毀則不須執行

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.models.convnext import convnext_tiny
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from tqdm import tqdm
import matplotlib.pyplot as plt
import dataset_module  # 自定義的數據集模組

In [ ]:
# 定義自定義數據轉換
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# 創建自定義數據集
train_dataset = dataset_module.YourCustomDataset(root='./img/train', transform=transform)
val_dataset = dataset_module.YourCustomDataset(root='./img/test', transform=transform)

In [ ]:
# 定義模型架構
model = convnext_tiny(pretrained=False).cuda()  # 不使用預訓練權重，根據自定義數據集的類別數量調整輸出層大小

# 定義損失函數和優化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [ ]:
import torch
import matplotlib.pyplot as plt

# 訓練與驗證的 DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

num_epochs = 10  # 訓練 epoch 數量

# 儲存每個 epoch 的訓練和驗證損失與準確率
train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    for images, labels in train_loader:
        images = images.cuda()
        labels = labels.cuda()
        
        optimizer.zero_grad()  # 重置梯度
        outputs = model(images)  # 前向傳播
        loss = criterion(outputs, labels)  # 計算損失
        loss.backward()  # 反向傳播
        optimizer.step()  # 更新參數
        
        running_train_loss += loss.item()  # 累加訓練損失

    train_losses.append(running_train_loss / len(train_loader))  # 計算平均訓練損失

    # 模型驗證階段
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.cuda()
            labels = labels.cuda()
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()  # 累加驗證損失
            _, predicted = torch.max(outputs, 1)  # 獲取預測標籤
            total += labels.size(0)
            correct += (predicted == labels).sum().cpu().item()  # 計算正確預測數

    val_losses.append(val_loss / len(val_loader))  # 計算平均驗證損失
    val_accuracies.append(100 * correct / total)  # 計算驗證準確率

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, Validation Loss: {val_losses[-1]:.4f}, Accuracy: {val_accuracies[-1]:.2f}%")

# 繪製圖表
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# 繪製損失圖
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Train Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 繪製準確率圖
plt.subplot(1, 2, 2)
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.title('Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
#儲存模型
torch.save(model.state_dict(), "./model.pth")